In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer


In [ ]:
df = pd.read_csv('tot.csv', encoding='cp949')
df.to_csv('tot.csv', encoding='cp949', index=False)

In [ ]:
# 자치구역 리스트 (총 17개)
regions = [
    '서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시',
    '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원특별자치도',
    '충청북도', '충청남도', '전북특별자치도', '전라남도', '경상북도', '경상남도', '제주특별자치도'
]

# 자치구역과 번호를 매핑
region_to_number = {region: idx + 1 for idx, region in enumerate(regions)}

# 데이터프레임에 새로운 열로 추가
df['자치구역번호'] = df['행정구역'].map(region_to_number)
df.drop(['행정구역'], axis=1, inplace=True)
df.to_csv('TOTAL.csv', encoding='cp949', index=False)


In [ ]:
df1 = pd.read_csv('earlybirth.csv', encoding='cp949')
df1_1 = df1.loc[(df1['행정구역별']!='전국') & (df1['종류별']=='조출생률(천명당)')]
df1_1.drop(['행정구역별','종류별', '항목', '단위'], axis=1, inplace=True)

df1_2 = df1_1.stack()
df1_2 = df1_2.reset_index(drop=True)

df['조출생률'] = df1_2


In [ ]:
df2 = pd.read_csv('highAge.csv', encoding='cp949')
df2_1 = df2.loc[(df2['행정구역별(1)']!='전국')]
df2_2 = df2_1.loc[((df2_1['시점']!=2008.01)& (df2_1['시점']!=2024.03)& (df2_1['시점']!=2024.04))]

df2_4 = df2_2['고령인구비율<br>(A÷B×100) (%)']
df2_5 = df2_4.reset_index(drop=True)

df['고령인구비율'] = df2_5


In [ ]:
df3 = pd.read_csv('2034move.csv', encoding='cp949')
df3.drop(0, axis=0, inplace=True)
df3_1 = df3.loc[(df3['행정구역(시군구)별']!='전국') & (df3['연령별']=='계')]
df3_2 = df3_1.iloc[:, 3:-17]

df3_5 = df3_2.stack()
df3_6 = df3_5.reset_index(drop=True)
df['청년 순이동수'] = df3_6

In [ ]:
df4 = pd.read_csv('company.csv', encoding='cp949')
df4_1 = df4['사업체수']
df4_2  = df4_1.replace('null', '-')
df['사업체수'] = df4_2

df.drop(df.index[1352:1406], inplace=True)
len(df)

In [1]:
df.replace('-', np.nan, inplace=True)
df.replace('', np.nan, inplace=True)


 # 결측값을 선형 회귀로 채우는 함수
def fill_missing_with_regression(df, column):
    not_null = df[df[column].notnull()]
    is_null = df[df[column].isnull()]
    if not_null.empty or is_null.empty:
        return df
    # X_train은 'Time'과 나머지 속성
    X_train = not_null[['시점', '조출생률', '고령인구비율', '청년 순이동수', '사업체수']].drop(columns=[column])
    y_train = not_null[column].values
    # X_test는 'Time'과 나머지 속성
    X_test = is_null[['시점', '조출생률', '고령인구비율', '청년 순이동수', '사업체수']].drop(columns=[column])
    # 결측값을 평균값으로 대체 (임시로)
    imputer = SimpleImputer(strategy='mean')
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.transform(X_test)
    # 선형 회귀 모델 훈련
    model = LinearRegression()
    model.fit(X_train, y_train)

    # 예측값으로 결측값 채우기
    predicted = model.predict(X_test)
    df.loc[df[column].isnull(), column] = predicted
    return df


# 데이터 정렬
df = df.sort_values(by=['자치구역번호', '시점'])

for column in ['조출생률', '고령인구비율', '청년 순이동수', '사업체수']:
    # 'DistrictNumber'로 그룹화하고 보간
    data_interpolated = df.groupby('자치구역번호').apply(lambda group: fill_missing_with_regression(group, column)).reset_index(drop=True)

# 보간된 데이터프레임을 CSV 파일로 저장
len(data_interpolated)

NameError: name 'df' is not defined

In [2]:
data = pd.read_csv('year_culturalFacilities_Academy_Indicator.csv', encoding='cp949')
data_interpolated['문화시설 개수'] = data['문화시설 개수']
data_interpolated['학원수 (개)'] = data['학원수 (개)']
data_interpolated['수강인원 (명)'] = data['수강인원 (명)']
data_interpolated['소멸위험지수'] = data['소멸위험지수']
data_interpolated['단계'] = data['단계']

data_interpolated.to_csv('final.csv', index=False)


NameError: name 'pd' is not defined

In [ ]:
data_interpolated.isna().sum()

In [ ]:
data_interpolated.fillna(method='ffill', inplace=True)
data_interpolated.isna().sum()

data_interpolated.head()